In [ ]:
options(repr.plot.width=6, repr.plot.height=4)

In [ ]:
library('ggplot2')
library('lme4')
str(diamonds)
data(diamonds)

In [ ]:
df <- data.frame(diamonds)
df <- df[sample(nrow(df), 1000),]

In [ ]:
g = ggplot(df, aes(x=carat, y=price))
g = g + geom_point(size=1, colour='black')
g = g + geom_point(size=1, colour='blue')
g

In [ ]:
y <- df$price; x <- df$carat;
fit <- lm(y ~ x)
r <- resid(fit)

In [ ]:
ggplot(diamonds, aes(r)) + geom_histogram(bins=100)

In [ ]:
ggplot(diamonds, aes(price, fill=cut)) + geom_histogram(binwidth = 500)


In [ ]:
str(diamonds)

In [ ]:
diamonds$e <- resid(fit)

In [ ]:
ggplot(diamonds[sample(nrow(diamonds), size=1000),], aes(x=price, y=e)) +
geom_hline(yintercept=0, size=0.2, color='red') +
geom_point(size=1, colour='black', alpha=0.3)

In [ ]:
df <- diamonds[sample(nrow(diamonds), 1000),]

In [ ]:
e = c(resid(lm(price~1, data=df)), resid(lm(price~carat, data=df)))
fit = factor(c(rep('itc', nrow(df)), rep('itc + slope', nrow(df))))

In [ ]:
ggplot(data.frame(e=e, fit=fit), aes(y=e, x=fit, fill=fit)) +
geom_dotplot(binaxis = 'y', size=0.5, stackdir = 'center', binwidth = 100)

### Manually making statistics for the inference

In [ ]:
y <- diamonds$price; x<-diamonds$carat; n <-length(y);

In [ ]:
b1 = cor(x,y)*sd(y)/sd(x); b0 = mean(y) - b1*mean(x)
ssx <- sum((x - mean(x))^2)
nu <- n - 2;
e <- (y - b0 - b1*x); 
s <- sqrt(sum(e^2)/nu)

se_b1 = s /sqrt(ssx); se_b0 = sqrt(1/n + mean(x)^2/ssx)*s
t_b0 = b0/se_b0; t_b1 = b1/se_b1
p_b0 = pt(abs(t_b0), df=nu, lower.tail = FALSE)
p_b1 = pt(abs(t_b1), df=nu, lower.tail = FALSE)

In [ ]:
quantile(e, c(0, .25, .5, .75, 1))

In [ ]:
coeff_table <- rbind(c(b0, se_b0, t_b0, p_b0), c(b1, se_b1, t_b1, p_b1))
colnames(coeff_table) <- c("Estimate", "Std. error", "t value", "P(>|t|)")
rownames(coeff_table) <- c("(Intercept)", "x")
coeff_table

In [ ]:
fit <- lm(y~x)

In [ ]:
summary(fit)

In [ ]:
summary_coefficients <- summary(fit)$coefficients
df <- summary(fit)$df

In [ ]:
summary_coefficients

In [ ]:
summary_coefficients[1,1] + c(-1, 1)*qt(0.975, df=df[2])*summary_coefficients[1,2]
summary_coefficients[2,1] + c(-1, 1)*qt(0.975, df=df[2])*summary_coefficients[2,2]

---

Week 2 test

#### q1
Consider the following data with x as the predictor and y as as the outcome.

```
x <- c(0.61, 0.93, 0.83, 0.35, 0.54, 0.16, 0.91, 0.62, 0.62)
y <- c(0.67, 0.84, 0.6, 0.18, 0.85, 0.47, 1.1, 0.65, 0.36)
```

Give a P-value for the two sided hypothesis test of whether β1 from a linear regression model is 0 or not.

In [ ]:
x <- c(0.61, 0.93, 0.83, 0.35, 0.54, 0.16, 0.91, 0.62, 0.62)
y <- c(0.67, 0.84, 0.6, 0.18, 0.85, 0.47, 1.1, 0.65, 0.36)
fit <- lm(y~x)
summary(fit)

### q2

Consider the previous problem, give the estimate of the residual standard deviation.

In [ ]:
e <- resid(fit)
n <- length(y)
s <- sqrt(sum(e^2)/(n-2))
s

### q3

In the 𝚖𝚝𝚌𝚊𝚛𝚜 data set, fit a linear regression model of weight (predictor) on mpg (outcome). Get a 95% confidence interval for the expected mpg at the average weight. What is the lower endpoint?

In [ ]:
g = ggplot(mtcars, aes(x=wt, y=mpg))
g = g + geom_point(size=1, colour='black')
g

In [ ]:
fit3 <- lm(mpg ~ wt, data = mtcars)

In [ ]:
n <- length(mtcars)
e <- resid(fit3)
s <- sqrt((sum(e**2)/(n - 2)))
s

In [ ]:
summary(fit3)

$$ \hat{y}_{sd} = \hat{\sigma}\sqrt{1+\frac{1}{n} + \frac{(x_0 - \bar{x})^2}{\sum{(x_i - \bar{x})^2}}}$$

In [ ]:
predict.lm(fit3, data.frame(wt=mean(mtcars$wt)), 
           df=summary(fit3)$df[2], interval='confidence', level = 0.95)

In [ ]:
20.09062 + c(-1,1)*qt(0.975, df=n-1)*s*(1/n)

#### q5

Consider again the 𝚖𝚝𝚌𝚊𝚛𝚜 data set and a linear regression model with mpg as predicted by weight (1,000 lbs). A new car is coming weighing 3000 pounds. Construct a 95% prediction interval for its mpg. What is the upper endpoint?

In [ ]:
predict(fit3, data.frame(wt=3), df=30, interval='confidence', level=0.95)[3]

#### q6

Consider again the 𝚖𝚝𝚌𝚊𝚛𝚜 data set and a linear regression model with mpg as predicted by weight (in 1,000 lbs). A “short” ton is defined as 2,000 lbs. Construct a 95% confidence interval for the expected change in mpg per 1 short ton increase in weight. Give the lower endpoint.

In [ ]:
fit <- lm(mpg ~ wt, data=mtcars)
n <- length(mtcars)
yhat <- coef(fit)[1] +  coef(fit)[2]*mtcars$wt
e <- mtcars$mpg - yhat
s <- sqrt(sum(e^2)/(n-2))
ssx <- sum((mtcars$wt - mean(mtcars$wt))^2)

In [ ]:
coef(fit)[2]+ c(-1, 1)*4*pt(0.975, df=summary(fit)$df[2], lower.tail=FALSE)*s/sqrt(ssx)

#### q9

Refer back to the mtcars data set with mpg as an outcome and weight (wt) as the predictor. About what is the ratio of the the sum of the squared errors, ∑ni=1(Yi−Y^i)2 when comparing a model with just an intercept (denominator) to the model with the intercept and slope (numerator)?

In [ ]:
fit1 <- lm(mpg ~ 1, data=mtcars)

fit2 <- lm(mpg ~ wt, data=mtcars)

yhat1 <- coef(fit1)[1]
yhat2 <- coef(fit2)[1] + coef(fit2)[2]*mtcars$wt

e1 <- mtcars$mpg - yhat1
e2 <- mtcars$mpg - yhat2

se1 = sum(e1^2)
se2 = sum(e2^2)


se1/se2